In [ ]:
import sample_generators
from sample_generators import makeEdges
from sample_generators import makeTree
from sample_generators import generate_equivalence_class
from tqdm.notebook import tqdm
import multiprocessing as mp
import time
import numpy as np 


errors = np.zeros(3)

def log_result(result):
    global pbar
    global errors
    pbar.update()
    errors = errors + result

def do_async(args, MAX_ITER):
    num_processors = 3
    p=mp.Pool(processes = num_processors)
    for i in range(MAX_ITER):
        p.apply_async(sample_generators.parallelTestFunc, args = args, callback = log_result) 
    p.close()
    p.join()


THETA = 0.1
Q = 0
NOISY_NODES = [1,3,5,7,9,11]
MAX_ITER = 10**3
ERROR_CUTOFF = 10**-3
STARTING_SAMPLES = 100
SAMPLE_STEP = 100
NODES = 12  #number of nodes

chainTree = [0]+list(range(NODES-1))
starTree = [0]*NODES
hybridTree = [0]+list(range(NODES//2-1))+[NODES//2-1]*(12//2-1)

edges = [(1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (7, 5), (8, 5), (9, 5), (10, 5), (11, 5)]
sample_range = []
num_samples = STARTING_SAMPLES

m_errors = []
cl_errors = []
k_errors = []

pbar = tqdm()
tqdm.refresh


m_error_probability = 1
tree = makeTree(edges)
equivalence_class = generate_equivalence_class(makeEdges(tree))

W = sample_generators.gen_chain_W(THETA,makeTree(edges))
prob = sample_generators.gen_prob(np.flip(W))

# for num_samples in sample_range:
while m_error_probability >= ERROR_CUTOFF:
    args = (num_samples, THETA, Q, NOISY_NODES, tree,equivalence_class, prob)
    errors = np.zeros(3)
    timei = time.time()
    pbar.reset(total = len(range(MAX_ITER)))
    
    if __name__ ==  '__main__': 
        do_async(args, MAX_ITER)
    pbar.refresh()
    errors = errors/MAX_ITER

    m_error_probability = errors[0]
    k_error_probability = errors[1]
    cl_error_probability = errors[2]

    m_errors.append(m_error_probability)
    k_errors.append(k_error_probability)
    cl_errors.append(cl_error_probability)

    sample_range.append(num_samples)

    elapsed = time.time()-timei
    print('samples:', num_samples,' m_error:',  m_error_probability, ' k_error:', k_error_probability, 'cl_error:', cl_error_probability, 'elapsed', elapsed)
    
    num_samples += SAMPLE_STEP
